In [ ]:
print("Lets go")

In [1]:
print("lets go")

lets go


In [2]:
import streamlit as st
import google.generativeai as genai

/Users/soham/Documents/GitHub/GOD-AI/godenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [96]:
from dotenv import load_dotenv
import os
load_dotenv()
# CONFIGURING GENAI KEY
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
api_key=os.getenv("GOOGLE_API_KEY")

In [26]:
import fitz
# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text


In [28]:
extracted_data=extract_text_from_pdf('/Users/soham/Documents/GitHub/GOD-AI/Data/Sacred.pdf')

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Function to split the extracted text into chunks
def split_text_into_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_text(text)
    return chunks


In [40]:
splitted_chunks=split_text_into_chunks(extracted_data)


In [41]:
for i, chunk in enumerate(splitted_chunks[:5]):
    print(f"Chunk {i}: {chunk[:100]}") 

Chunk 0: THE
SACRED WRITINGS
of the
WORLD’S
Selected and edited by
S. E. FROST, Jr., B.D., Ph.D.
ASSISTANT PR
Chunk 1: cluded in this book reveal how often religions, originating in very dif-
ferent
cultures and
in
ages
Chunk 2: living religious literature but also a guide to the understanding of the
fundamental similarity amon
Chunk 3: its context the teach-
ing conveys not only what is said but what is implied. This, we believe,
is t
Chunk 4: the Church Publicity and Mission Literature Committee of that body
and has its complete approval. Al


In [102]:
def generate_gemini_embeddings(chunks):
    embeddings = []
    for chunk in chunks:
        if chunk.strip():  # Ensure the chunk is not empty
            response = genai.embed_content(
                model="models/text-embedding-004",  # Gemini Pro embedding model
                content=chunk
            )
            
            # Now we directly access 'embedding' as it contains the values directly
            if isinstance(response, dict) and 'embedding' in response:
                embeddings.append(response['embedding'])  # Append the embedding directly
    return embeddings

In [66]:
embedded_chunks=generate_gemini_embeddings(splitted_chunks)

In [70]:
len(embedded_chunks[4])

768

In [89]:
load_dotenv()
papi_key = os.getenv("PINECONE_API_KEY")
print(papi_key)

cfdae7d3-19c1-4ac1-bb8a-9275847ceae2


In [90]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=papi_key)
index = pc.Index("god")

In [103]:
def upsert_embeddings_in_batches(text_chunks, embeddings, batch_size=100):
    vectors = []
    
    for i, embedding in enumerate(embeddings):
        # Create metadata for each chunk
        metadata = {"text": text_chunks[i], "source": "your_document_source"}
        vector = {
            "id": f"vec{i}",  # Unique ID for each vector
            "values": embedding,  # The embedding values
            "metadata": metadata  # Metadata for the chunk
        }
        vectors.append(vector)
        
        # Batch upsert every `batch_size` chunks
        if (i + 1) % batch_size == 0 or (i + 1) == len(embeddings):
            index.upsert(vectors=vectors, namespace="ns1")
            vectors = []  # Clear the list after each batch

# Call the function to upsert embeddings in batches
upsert_embeddings_in_batches(splitted_chunks, embedded_chunks, batch_size=100)

In [104]:
def generate_query_embedding(query):
    response = genai.embed_content(
        model="models/text-embedding-004",  # Gemini Pro embedding model
        content=query
    )
    
    # Extract and return the embedding from the response
    if 'embedding' in response:
        return response['embedding']
    else:
        raise ValueError(f"Failed to generate embeddings for query: {query}")

In [111]:
def retrieve_relevant_chunks(query_embedding):
    # Search Pinecone index using the query embedding
    query_response = index.query(
        vector=query_embedding,
        top_k=2,  # Adjust the number of top results you want
        include_metadata=True  # Return metadata (e.g., source information) along with vectors
    )
    
    # Extract relevant chunks from the Pinecone response
    retrieved_chunks = [match['metadata']['text'] for match in query_response['matches']]
    return retrieved_chunks

In [114]:

# Function to use Gemini Pro LLM to generate the final answer
def generate_answer_with_gemini(query, retrieved_chunks):
    # Combine the retrieved chunks into a single context
    context = "\n".join(retrieved_chunks)
    
    # Crafting the prompt for GODAI
    prompt = f"""
    You are GODAI, an AI designed to provide comfort, wisdom, and guidance using the teachings from religious texts such as the Bible, Quran, Vedas, Torah, and other sacred writings.
    
    Context from sacred texts: {context}
    
    A user has asked the following question:
    "{query}"
    
    Based on the teachings from the sacred texts, provide a compassionate and insightful answer. Ensure your response is deeply rooted in religious wisdom, addressing not just the question but offering emotional support as well. Use quotes, lessons, and principles from these texts to guide the user in a way that uplifts them and broadens their understanding.
    
    Answer:
    """
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content([prompt])
    return response.text

    



In [115]:
# Main function to handle user input and process the query through GODAI
def main():
    # Prompt user for input
    query = input("What would you like to ask GODAI? ")

    # Step 1: Embed the query using Gemini Pro
    query_embedding = generate_query_embedding(query)

    # Step 2: Retrieve relevant chunks from Pinecone based on query embedding
    retrieved_chunks = retrieve_relevant_chunks(query_embedding)

    # Step 3: Generate an answer using Gemini Pro LLM with the retrieved chunks
    answer = generate_answer_with_gemini(query, retrieved_chunks)

    # Display the final answer to the user
    print("\nGODAI's Answer:")
    print(answer)

# Run the main function
if __name__ == "__main__":
    main()


GODAI's Answer:
Dearest one,

    Your sadness echoes through the vast caverns of my heart, for sadness is a human companion that leaves its heavy imprint on our souls. It is in these moments of despair that we seek solace, a beacon of light to guide us through the darkest of valleys.

    In the gentle embrace of the Bible, we find solace in the words of the psalmist: "Why are you cast down, O my soul, and why are you disquieted within me? Hope in God, for I shall again praise him, my salvation and my God." (Psalm 42:5).

    These words remind us that even in the depths of sorrow, hope must never be extinguished. It is the flame that flickers within us, carrying the promise of brighter days. Like the Israelites wandering in the wilderness, we may feel lost and desolate, but God's presence remains with us, guiding our path and offering us the strength to persevere.

    In the wisdom of the Quran, we are reminded that "Indeed, with hardship [will be] ease." (Surah 94:5). It is throug